# 13 - Hybrid Functionals

This tutorial demonstrates hybrid functional calculations (HSE06)
for accurate band gap prediction, using silicon as an example.

**Note:** Hybrid functional calculations are computationally expensive!
Using reduced settings for demonstration.

In [1]:
import numpy as np
from ase.build import bulk

from vasp import Vasp
from vasp.parameters import get_hybrid_params

print("=" * 60)
print("Hybrid Functional Calculations")
print("=" * 60)
print()

print("Note: Hybrid calculations are 10-100x more expensive than GGA!")
print("      Using coarse settings for demo.")
print()

Hybrid Functional Calculations

Note: Hybrid calculations are 10-100x more expensive than GGA!
      Using coarse settings for demo.



## Part 1: Silicon with PBE (reference)

In [8]:
print("Part 1: Silicon band gap - PBE reference")
print("-" * 40)
print()

si = bulk('Si', 'diamond', a=5.43)

calc_pbe = Vasp(
    label='results/hybrid/si_pbe',
    atoms=si,
    xc='PBE',
    encut=300,  # Reduced for demo
    kpts=(4, 4, 4),  # Reduced for demo
    ismear=0,
    sigma=0.05,
)

e_pbe = calc_pbe.potential_energy

print("Silicon with PBE:")
print(f"  Total energy: {e_pbe:.6f} eV")

Part 1: Silicon band gap - PBE reference
----------------------------------------

Silicon with PBE:
  Total energy: -10.835611 eV


## Part 2: Silicon with HSE06

In [9]:
print("Part 2: Silicon band gap - HSE06")
print("-" * 40)
print()

# Get HSE06 parameters
hse_params = get_hybrid_params('hse06')

print("HSE06 parameters:")
print("  LHFCALC = True (enable hybrid)")
print("  HFSCREEN = 0.2 (screening, 1/A)")
print("  AEXX = 0.25 (25% exact exchange)")
print()

calc_hse = Vasp(
    label='results/hybrid/si_hse',
    atoms=si,
    encut=300,  # Reduced for demo
    kpts=(3, 3, 3),  # Very coarse for speed
    ismear=0,
    sigma=0.05,
    **hse_params,
)

e_hse = calc_hse.potential_energy

print("Silicon with HSE06:")
print(f"  Total energy: {e_hse:.6f} eV")

Part 2: Silicon band gap - HSE06
----------------------------------------

HSE06 parameters:
  LHFCALC = True (enable hybrid)
  HFSCREEN = 0.2 (screening, 1/A)
  AEXX = 0.25 (25% exact exchange)

Silicon with HSE06:
  Total energy: -12.012787 eV


## Part 3: Different hybrid functionals

In [4]:
print("Part 3: Hybrid functional options")
print("-" * 40)
print()

print("Available hybrid functionals:")
print()

functionals = [
    ('pbe0', 'PBE0', '25% HF, no screening'),
    ('hse06', 'HSE06', '25% HF, screened (0.2 A^-1)'),
    ('hse03', 'HSE03', '25% HF, screened (0.3 A^-1)'),
    ('b3lyp', 'B3LYP', '20% HF, empirical mix'),
]

for key, name, desc in functionals:
    params = get_hybrid_params(key)
    print(f"  {name}:")
    print(f"    {desc}")
    print(f"    AEXX={params.get('aexx', 0.25)}")
    print()

Part 3: Hybrid functional options
----------------------------------------

Available hybrid functionals:

  PBE0:
    25% HF, no screening
    AEXX=0.25

  HSE06:
    25% HF, screened (0.2 A^-1)
    AEXX=0.25

  HSE03:
    25% HF, screened (0.3 A^-1)
    AEXX=0.25

  B3LYP:
    20% HF, empirical mix
    AEXX=0.2



## Part 4: Computational considerations

In [5]:
print("Part 4: Computational considerations")
print("-" * 40)
print()

print("Hybrid functionals are expensive because:")
print("  - Exact exchange requires evaluating 4-center integrals")
print("  - Scales as O(N^4) with system size")
print("  - Each k-point interacts with all others")
print()

print("Strategies to reduce cost:")
print()
print("  1. Start from PBE:")
print("     - Relax structure with PBE first")
print("     - Use ISTART=1, ICHARG=1 to read PBE data")
print()
print("  2. Use PRECFOCK = Fast:")
print("     - Coarser FFT grid for HF")
print("     - Minor accuracy loss, significant speedup")
print()
print("  3. Use NKRED:")
print("     - Reduce k-points for HF part only")
print("     - Set NKRED = 2 to halve HF k-mesh")
print()

Part 4: Computational considerations
----------------------------------------

Hybrid functionals are expensive because:
  - Exact exchange requires evaluating 4-center integrals
  - Scales as O(N^4) with system size
  - Each k-point interacts with all others

Strategies to reduce cost:

  1. Start from PBE:
     - Relax structure with PBE first
     - Use ISTART=1, ICHARG=1 to read PBE data

  2. Use PRECFOCK = Fast:
     - Coarser FFT grid for HF
     - Minor accuracy loss, significant speedup

  3. Use NKRED:
     - Reduce k-points for HF part only
     - Set NKRED = 2 to halve HF k-mesh



## Part 5: When to use hybrids

In [6]:
print("Part 5: When to use hybrid functionals")
print("-" * 40)
print()

print("Use hybrid functionals when:")
print("  - Accurate band gaps are needed")
print("  - Band alignment is important")
print("  - Charge localization matters")
print("  - Defect levels in semiconductors")
print()

print("Standard GGA is often sufficient for:")
print("  - Geometry optimization")
print("  - Relative energies (if error cancels)")
print("  - Metallic systems")
print("  - Initial screening")
print()

Part 5: When to use hybrid functionals
----------------------------------------

Use hybrid functionals when:
  - Accurate band gaps are needed
  - Band alignment is important
  - Charge localization matters
  - Defect levels in semiconductors

Standard GGA is often sufficient for:
  - Geometry optimization
  - Relative energies (if error cancels)
  - Metallic systems
  - Initial screening



## Summary

In [7]:
print("=" * 60)
print("Summary")
print("=" * 60)
print()

print("Band gap comparison (literature values):")
print()
print(f"  {'Material':<10} {'PBE (eV)':<12} {'HSE06 (eV)':<12} {'Exp (eV)':<12}")
print("  " + "-" * 46)
print(f"  {'Si':<10} {'~0.6':<12} {'~1.1':<12} {'1.12':<12}")
print(f"  {'GaAs':<10} {'~0.4':<12} {'~1.4':<12} {'1.42':<12}")
print(f"  {'ZnO':<10} {'~0.7':<12} {'~2.5':<12} {'3.37':<12}")
print()

print("Demo settings used:")
print("  - ENCUT = 300 eV")
print("  - kpts = 3x3x3 (HSE), 4x4x4 (PBE)")
print("  - Si only (skipped GaAs for speed)")
print()

print("For production:")
print("  - ENCUT 400+ eV")
print("  - Dense k-points (6x6x6 or more)")
print("  - Consider PRECFOCK=Fast, NKRED=2")
print()
print("Next: Try 14_van_der_waals/ for dispersion corrections.")

Summary

Band gap comparison (literature values):

  Material   PBE (eV)     HSE06 (eV)   Exp (eV)    
  ----------------------------------------------
  Si         ~0.6         ~1.1         1.12        
  GaAs       ~0.4         ~1.4         1.42        
  ZnO        ~0.7         ~2.5         3.37        

Demo settings used:
  - ENCUT = 300 eV
  - kpts = 3x3x3 (HSE), 4x4x4 (PBE)
  - Si only (skipped GaAs for speed)

For production:
  - ENCUT 400+ eV
  - Dense k-points (6x6x6 or more)
  - Consider PRECFOCK=Fast, NKRED=2

Next: Try 14_van_der_waals/ for dispersion corrections.
